In [3]:
using DataFrames
using JuMP, Gurobi

In [4]:
# number of teams
N = 20
# number of weeks
W = N-1
# number of days per weeks
D = 7;

In [5]:
function GetC()
    C = zeros((D*W, D*W))

    for t in 1:(W*D)
        for t_ in 1:(W*D)
            if t < t_
                C[t, t_] = (t_ - t)^(-1)
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [6]:
function Optimize4WeeksAndFreeze(X_old, Y_old)

    # first time, X_old and Y_old initialized to empty
    X_old = Array{Float64}(undef, 0, 0)
    Y_old = Array{Float64}(undef, 0, 0)

    # create model
    model = Model(Gurobi.Optimizer)

    # set time limit
    set_model_attribute(model, "TimeLimit", 30)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    W = 4 # weeks
    D = 7 # days

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = 1:W, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = 1:W, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j on w, 0 otherwise

    # OBJECTIVE FUNCTION
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in 1:W-1, d in 1:D, d_ in 1:D))

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d] - 1)
    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = 1:W, d = 1:D], x[i, i, w, d] == 0)
    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 1:D], x[i, j, w, d] == x[j, i, w, d])
    # each team can only play one game per week
    @constraint(model, [i = 1:N, w = 1:W], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)
    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in 1:W, d in 1:D) == W)
    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in 1:W, d in 1:D) <= 1)
    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = 1:W))
    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in 1:W, d in 1:D) <= 1 - Y_old[i, j])
    


    # OPTIMIZE


    # update old X, Y before returning them


end


ErrorException: syntax: incomplete: "function" at /Users/andreazanon/Optimization_Project/Optimize4WeeksForAllTeams_FreezeAndMoveWindow.ipynb:1 requires end